In [1]:
import re

import pandas as pd
import numpy as np
import os

import seaborn as sns


## Concatenate Dataset into one File

In [99]:
from typing import List


def eraseSpace(directory: str):
    files = getFileNameFromDir(direc=directory)
    for file in files:
        realFile = open("{}/{}".format(directory, file), mode="r", encoding="UTF-8")
        data = [f for f in realFile if f.strip()]
        realFile.close()
        realFile = open("{}/{}".format(directory, file), mode="w", encoding="UTF-8")
        realFile.writelines(data)
        realFile.close()


def getFileNameFromDir(direc: str) -> List[str]:
    return [f for f in os.listdir("{}/".format(direc))]


def myConcat(direcToScan: str, outputFilename: str, outputDirec: str = "all_data"):
    files = getFileNameFromDir(direc=direcToScan)
    datasets = pd.concat([pd.read_csv("{}/{}".format(direcToScan, f)) for f in files], axis=0, ignore_index=True)

    datasets.to_csv("{}/{}.csv".format(outputDirec, outputFilename), index=False)

In [100]:
## Concatenate All Dataset into one File
myConcat(direcToScan="datasets", outputFilename="all_data_dataset")

## Concatenate All Entity-tag Dataset into one File
myConcat(direcToScan="entity-tag", outputFilename="all_entity_dataset")

## Concatenate All Segmentation Dataset into one File
myConcat(direcToScan="segments", outputFilename="all_segment_dataset")

## Concatenate All Segmentation Dataset into one File
myConcat(direcToScan="POS/", outputFilename="all_pos_dataset")

## Erase blank from files

In [101]:
# try:
#     eraseSpace(directory="datasets")
#     eraseSpace(directory="entity-tag")
#     eraseSpace(directory="POS")
#     eraseSpace(directory="segments")
#     print("Success")
# except:
#     print("fail")


## Compare Ner length, POS length, and SEG Length

In [103]:
allEntityDataset = pd.read_csv("all_data/all_entity_dataset.csv")
allPosDataset = pd.read_csv("all_data/all_pos_dataset.csv")
allSegDataset = pd.read_csv("all_data/all_segment_dataset.csv")

# error = [1 for seg, entity, ent, pos in
#          zip(allSegDataset.items(), allEntityDataset["Question"].items(), allPosDataset["Question"].items()) if
#          len(seg) != len(ent) != len(pos)]
myDict = {}
indexs = allEntityDataset.index.tolist()
segs = allSegDataset["Question"].to_list()
tags = allSegDataset["Intent"].to_list()
ents = allEntityDataset["Question"].to_list()
poss = allPosDataset["Question"].to_list()

for seg, tag, ent, pos, index in zip(segs, tags, ents, poss, indexs):
    if len(str(seg).split()) is not len(str(pos).split()) is not len(str(ent).split()):
        myDict.setdefault("index", []).append(index)
        myDict.setdefault("intent", []).append(tag)
        myDict.setdefault("length_seg", []).append(len(str(seg).split()))
        myDict.setdefault("length_pos", []).append(len(str(pos).split()))
        myDict.setdefault("length_ent", []).append(len(str(ent).split()))
        myDict.setdefault("seg", []).append(seg)
        myDict.setdefault("pos", []).append(pos)
        myDict.setdefault("ent", []).append(ent)

print("length_seg: ", len(myDict["length_seg"]))
print("length_pos: ", len(myDict["length_pos"]))
print("length_ent: ", len(myDict["length_ent"]))
print("seg: ", len(myDict["seg"]))
print("pos: ", len(myDict["pos"]))
print("ent: ", len(myDict["ent"]))

# errorPd = pd.DataFrame.from_dict(myDict)
# # errorPd = errorPd.transpose()
# errorPd.to_csv("all_data/diff_length_sentences.csv")
# errorPd
print(myDict)

KeyError: 'length_seg'

## Find Amount of Sentences per intent as well as Total

In [ ]:
## Find Amount of Sentences
allDataset = pd.read_csv("all_data/all_data_dataset.csv")
allDataset.head()

In [ ]:
allDataset.groupby(allDataset["Intent"]).count()

In [ ]:
# Find Total
print("Total Sentence length: ", allDataset.Question.count())

## Check amount of entities as well as for any errors

In [ ]:
entityDataFrame = pd.read_csv("all_data/all_entity_dataset.csv")
entityFrame = pd.read_csv("Named Entity.csv")

In [ ]:
entityFrame.info()

In [ ]:
# Remove white space from column name and apply it to csv file
entityFrame = entityFrame.rename(columns={entityFrame.columns.values[1]: entityFrame.columns.values[1].strip()})
entityFrame.to_csv("Named Entity.csv")

In [ ]:
myDict = {}
entityTag = entityFrame["NE Code"].to_list()
entitySentences = entityDataFrame["Question"].to_list()

for x in entityTag:
    count = 0
    for j in entitySentences:
        sentence = j.split()
        for c in range(len(sentence)):
            # Matches /B-Word until space
            if re.search(r"/B-{}\w*".format(x), sentence[c]):
                count += 1
                split = sentence[c].rsplit("/B-", maxsplit=1)[1]
                myDict.update({split: count})

entityAmount = pd.DataFrame(myDict.items(), columns=["Intent", "Amount"])
# entityAmount
myDict

In [ ]:
## Calculate the percentage

# Sum of Intent
totalIntentTag = sum(myDict[x] for x in myDict)
rise = ["{0:.2f}%".format(myDict[x] * 100 / totalIntentTag) for x in myDict]

# array = np.asarray(rise)
#
# arrayV2 = np.fromiter(("{0:.2f}%".format(myDict[x]*100/totalIntentTag) for x in myDict), dtype=str)

entityAmount["Percentage"] = rise
entityAmount

## Count each word occurence

In [ ]:
# TODO: DO THIS

# myDict = {}
#
# for x in entityTag:
#     for j in entitySentences:
#         for i in j.split():
#             if re.search(r"/", i):
#                 word, entity = [split for split in i.split("/", maxsplit=1)]
#


## Find Different sentence length

In [ ]:
segmentData = pd.read_csv("all_data/all_segment_dataset.csv")
posData = pd.read_csv("all_data/all_pos_dataset.csv")
posData.drop(columns=["Intent"], inplace=True)
posData.rename(columns={"Intents": "Intent"}, inplace=True)

different_length = [1 if len(str(input)) != len(str(output)) else 0 for input, output in
                    zip(segmentData["Question"].tolist(), posData["Question"].tolist())]
print("{} sentences have different length.".format(sum(different_length)))

In [ ]:
mydict = {}
mydict.update({"intent": 1})

## Count entity tag

In [25]:
word1 = "B-ORG"
word2 = 'I-ORG'
count1 = 0
count2 = 0
with open("entity-tag/AskAboutCompetition.csv", 'r') as f:
    for line in f:
        line = line.replace("/B", " B" )
        line = line.replace("/I", " I" )
        words = line.split()
        for i in words:
            if(i==word1):
                count1=count1+1
            if(i==word2):
                count2=count2+1
        
print("Occurrences of  ", word1, ":", count1)
print("Occurrences of  ", word2, ":", count2)



Occurrences of   B-ORG : 120
Occurrences of   I-ORG : 180
